## Prepare Notebook

In [ ]:
import os
import arviz as az
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import numpy as np
import pandas as pd
import pymc as pm
import pymc.sampling_jax
import pytensor.tensor as pt
import seaborn as sns

from linearmodels.iv import IV2SLS

plt.style.use("bmh")
plt.rcParams["figure.figsize"] = [12, 7]
plt.rcParams["figure.dpi"] = 100
plt.rcParams["figure.facecolor"] = "white"

%load_ext autoreload
%autoreload 2
%config InlineBackend.figure_format = "retina"

In [ ]:
seed: int = sum(map(ord, "double_ml_bart"))
rng: np.random.Generator = np.random.default_rng(seed=seed)
random_seed_int: int = rng.integers(low=0, high=100, size=1).item()

## Read Data

In [ ]:
root_path = "https://raw.githubusercontent.com/matheusfacure/python-causality-handbook/master/causal-inference-for-the-brave-and-true/data/"
data_path = os.path.join(root_path, "app_engagement_push.csv")

df = pd.read_csv(data_path)

n = df.shape[0]

df.head()

## EDA

In [ ]:
fig, ax = plt.subplots(
    nrows=2, ncols=1, figsize=(9, 7), sharex=True, sharey=True, layout="constrained"
)
sns.histplot(x="in_app_purchase", hue="push_assigned", kde=True, data=df, ax=ax[0])
sns.histplot(x="in_app_purchase", hue="push_delivered", kde=True, data=df, ax=ax[1])
fig.suptitle("Histogram of in-app purchases", fontsize=16);

In [ ]:
df.groupby("push_assigned").agg({"in_app_purchase": ["mean", "std"]})

In [ ]:
ols_formula = "in_app_purchase ~ 1 + push_assigned"
ols = IV2SLS.from_formula(formula=ols_formula, data=df).fit()
ols.summary.tables[1]

In [ ]:
df.groupby("push_delivered").agg({"in_app_purchase": ["mean", "std"]})

In [ ]:
ols_formula = "in_app_purchase ~ 1 + push_delivered"
ols = IV2SLS.from_formula(formula=ols_formula, data=df).fit()
ols.summary.tables[1]

In [ ]:
iv_formula = "in_app_purchase ~ 1 + [push_delivered ~ push_assigned]"
iv = IV2SLS.from_formula(formula=iv_formula, data=df).fit()
iv.summary.tables[1]

In [ ]:
y = df["in_app_purchase"].to_numpy()
t = df["push_delivered"].to_numpy()
z = df["push_assigned"].to_numpy()

In [ ]:
with pm.Model() as model:
    intercept_y = pm.Normal(name="intercept_y", mu=50, sigma=10)
    intercept_t = pm.Normal(name="intercept_t", mu=0, sigma=1)
    beta_t = pm.Normal(name="beta_t", mu=0, sigma=10)
    beta_z = pm.Normal(name="beta_z", mu=0, sigma=10)
    sd_dist = pm.HalfCauchy.dist(beta=2, shape=2)
    chol, corr, sigmas = pm.LKJCholeskyCov(name="chol_cov", eta=2, n=2, sd_dist=sd_dist)

    pm.Deterministic(name="cov", var=pt.dot(l=chol, r=chol.T))
    
    mu_y = pm.Deterministic(name="mu_y", var=beta_t * t + intercept_y)
    mu_t = pm.Deterministic(name="mu_t", var=beta_z * z + intercept_t)
    mu = pm.Deterministic(name="mu", var=pt.stack(tensors=(mu_y, mu_t), axis=1))
    
    likelihood = pm.MvNormal(
        name="likelihood",
        mu=mu,
        chol=chol,
        observed=np.stack(arrays=(y, t), axis=1),
        shape=(n, 2),
    )

pm.model_to_graphviz(model=model)


In [ ]:
with model:
    idata = pm.sampling_jax.sample_numpyro_nuts(draws=4_000, chains=4, random_seed=rng)

In [ ]:
var_names = ["beta_t", "beta_z", "intercept_y", "intercept_t"]

az.summary(data=idata, var_names=var_names)

In [ ]:
axes = az.plot_trace(
    data=idata,
    var_names=var_names,
    compact=True,
    kind="rank_bars",
    backend_kwargs={"figsize": (10, 7), "layout": "constrained"},
)
plt.gcf().suptitle("IV Model - Trace", fontsize=16);

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))
hdi_prob = 0.94
az.plot_posterior(
    data=idata,
    var_names=["beta_t"],
    ref_val=iv.params["push_delivered"],
    hdi_prob=hdi_prob,
    ax=ax,
)
ax.axvline(
    x=iv.conf_int(level=hdi_prob).loc["push_delivered", "lower"],
    color="C1",
    ls="--",
    lw=1,
    label=f"{hdi_prob: .0%} CI (lower)",
)
ax.axvline(
    x=iv.conf_int(level=hdi_prob).loc["push_delivered", "upper"],
    color="C1",
    ls="--",
    lw=1,
    label=f"{hdi_prob: .0%} CI (upper)",
)
ax.legend()
ax.set(title="IV Model - Posterior Distribution Effect");

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))
sns.histplot(
    x=az.extract(data=idata, var_names=["chol_cov_corr"])[0, 1, :],
    kde=True,
    color="C2",
    ax=ax,
)
ax.set(title="IV Model - Posterior Distribution Correlation")


In [ ]:
cov_mean = idata.posterior["cov"].mean(dim=("chain", "draw"))

cov_samples = np.random.multivariate_normal(
    mean=np.zeros(shape=(2)), cov=cov_mean, size=5_000
)

g = sns.jointplot(x=cov_samples[:, 0], y=cov_samples[:, 1], kind="kde", fill=True, height=6)
g.fig.suptitle("IV Model - Covariance Samples", fontsize=16, y=1.05);

---

## Hierarchical Model: Adding past experiments as priors

In [ ]:
b_j = np.array([3.4, 3.6, 4.25, 4.0, 3.9])
se_j = np.array([0.5, 0.55, 0.6, 0.45, 0.44])

In [ ]:
with pm.Model() as hierarchical_model:
    beta_t_hat = pm.Normal(name="beta_t_hat", mu=4, sigma=2)
    sigma_t_hat = pm.HalfCauchy(name="sigma_t_hat", beta=2)

    intercept_y = pm.Normal(name="intercept_y", mu=50, sigma=10)
    intercept_t = pm.Normal(name="intercept_t", mu=0, sigma=1)

    beta_z = pm.Normal(name="beta_z", mu=0, sigma=10)
    sd_dist = pm.HalfCauchy.dist(beta=2, shape=2)
    chol, corr, sigmas = pm.LKJCholeskyCov(name="chol_cov", eta=2, n=2, sd_dist=sd_dist)

    pm.Deterministic(name="cov", var=pt.dot(l=chol, r=chol.T))

    z_j = pm.Normal(name="z_j", mu=0, sigma=1, shape=b_j.size)
    beta_j = pm.Deterministic(name="beta_j", var=beta_t_hat + sigma_t_hat * z_j)
    pm.Normal(name="beta_j_observed", mu=beta_j, sigma=se_j, observed=b_j)

    w = pm.Normal(name="w", mu=0, sigma=1)
    beta_t = pm.Deterministic(name="beta_t", var=beta_t_hat + sigma_t_hat * w)
    
    mu_y = pm.Deterministic(name="mu_y", var=beta_t * t + intercept_y)
    mu_t = pm.Deterministic(name="mu_t", var=beta_z * z + intercept_t)
    mu = pm.Deterministic(name="mu", var=pt.stack(tensors=(mu_y, mu_t), axis=1))
    
    likelihood = pm.MvNormal(
        name="likelihood",
        mu=mu,
        chol=chol,
        observed=np.stack(arrays=(y, t), axis=1),
        shape=(n, 2),
    )

pm.model_to_graphviz(model=hierarchical_model)

In [ ]:
with hierarchical_model:
    hierarchical_idata = pm.sampling_jax.sample_numpyro_nuts(
        target_accept=0.95, draws=4_000, chains=4, random_seed=rng
    )

In [ ]:
print(f"Divergences = {hierarchical_idata.sample_stats.diverging.sum().item()}")

In [ ]:
var_names = [
    "beta_t_hat",
    "sigma_t_hat",
    "beta_t",
    "beta_z",
    "intercept_y",
    "intercept_t",
]

az.summary(data=hierarchical_idata, var_names=var_names)

In [ ]:
axes = az.plot_trace(
    data=hierarchical_idata,
    var_names=var_names,
    compact=True,
    kind="rank_bars",
    backend_kwargs={"figsize": (10, 9), "layout": "constrained"},
)
plt.gcf().suptitle("IV Model - Trace", fontsize=16);

In [ ]:
fig, ax = plt.subplots(
    nrows=2, ncols=1, figsize=(9, 7), sharex=True, sharey=False, layout="constrained"
)
hdi_prob = 0.94

az.plot_posterior(
    data=idata,
    var_names=["beta_t"],
    ref_val=iv.params["push_delivered"],
    hdi_prob=hdi_prob,
    ax=ax[0],
)
ax[0].axvline(
    x=iv.conf_int(level=hdi_prob).loc["push_delivered", "lower"],
    color="C1",
    ls="--",
    lw=1,
    label=f"{hdi_prob: .0%} CI (lower)",
)
ax[0].axvline(
    x=iv.conf_int(level=hdi_prob).loc["push_delivered", "upper"],
    color="C1",
    ls="--",
    lw=1,
    label=f"{hdi_prob: .0%} CI (upper)",
)
ax[0].legend()
ax[0].set(title="IV Model - Posterior Distribution Effect")

az.plot_posterior(
    data=hierarchical_idata,
    var_names=["beta_t"],
    hdi_prob=hdi_prob,
    ax=ax[1],

)

for j, (b, se) in enumerate(zip(b_j, se_j)):
    ax[1].axvline(x=b, color=f"C{j + 1}", ls="--", lw=1, label=f"b_{j}")

ax[1].legend(title="previous experiemnts", loc="upper left")
ax[1].set(title="IV Hierarchical Model - Posterior Distribution Effect");

In [ ]:
ax, *_ = az.plot_forest(
    data=hierarchical_idata,
    var_names=["beta_t_hat", "beta_t"],
    combined=True,
)

ax.axvline(x=iv.params["push_delivered"], color="C0", ls="--", label="IV")

for j, (b, se) in enumerate(zip(b_j, se_j)):
    ax.axvline(x=b, color=f"C{j + 1}", ls="--", lw=1, label=f"b_{j}")

ax.legend(loc="center left",  bbox_to_anchor=(1, 0.5))